In [1]:
# -*- coding: utf-8 -*-
from __future__ import unicode_literals
# text in Western (Windows 1252)

import pickle
import numpy as np
# import StringIO
import math
from keras import optimizers, metrics
from keras.models import Model
from keras.layers import Dense, Dropout, Input
from keras.layers.merge import concatenate
from keras import regularizers
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from keras.constraints import maxnorm
from keras.layers import Flatten
from keras.optimizers import SGD
from keras.models import load_model
# from keras import backend as Input
np.random.seed(7)

Using Theano backend.


In [2]:
# %run ../../../prepare_data.py

import sys
sys.path.insert(0, '../../../')
from prepare_data import *

In [3]:
# %run ../../../prepare_data.py
# X_train, X_other_features_train, y_train, X_test, X_other_features_test, y_test, X_validate, X_other_features_validate, y_validate = generate_syllable_inputs('../../internal_representations/inputs/content_shuffle_vector.h5', '../../internal_representations/inputs/shuffle_vector')
# save_inputs('../../internal_representations/inputs/shuffeled_syllable_train_inputs.h5', X_train, y_train, other_features = X_other_features_train)
# save_inputs('../../internal_representations/inputs/shuffeled_syllable_test_inputs.h5', X_test, y_test, other_features = X_other_features_test)
# save_inputs('../../internal_representations/inputs/shuffeled_syllable_validate_inputs.h5', X_validate, y_validate,  other_features = X_other_features_validate)
# X_train, X_other_features_train, y_train = load_inputs('../../internal_representations/inputs/shuffeled_syllable_train_inputs.h5', other_features=True)
# X_test, X_other_features_test, y_test = load_inputs('../../internal_representations/inputs/shuffeled_syllable_test_inputs.h5', other_features=True)
# X_validate, X_other_features_validate, y_validate = load_inputs('../../internal_representations/inputs/shuffeled_syllable_validate_inputs.h5', other_features=True)

# syllable_letters_translator = create_syllable_letters_translator(max_syllable, syllable_dictionary, dictionary, vowels)

data = Data('sl', accent_classification=True)
data.generate_data('syllabled_letters_accent_classification_train',
                   'syllabled_letters_accent_classification_test',
                   'syllabled_letters_accent_classification_validate', force_override=False)

READING CONTENT...
CONTENT READ SUCCESSFULLY
CREATING DICTIONARY...
DICTIONARY CREATION SUCCESSFUL!
GENERATING X AND y...
SHUFFELING INPUTS...
INPUTS SHUFFELED!
SHUFFELING INPUTS...
INPUTS SHUFFELED!
SHUFFELING INPUTS...
INPUTS SHUFFELED!
GENERATION SUCCESSFUL!


In [8]:
%run ../../../prepare_data.py
gen = data.generator('train', 16)
test = next(gen)

In [21]:
# prints test word from generator
# print(X_train[0])
# print(syllable_dictionary[test[0][0][0][0].tolist().index(1)])
print(np.array(test[0][0]).shape)
for el in test[0][0][0]:
    syllable = ''
    for pos in range(0, len(el), 36):
        syllable += data.decode_x([el[pos:pos+36]], dictionary)
    print(syllable)
#     print(el)
    
#     print(syllable_dictionary[el.tolist().index(1)])

(16, 11, 252)
na
ja
ktu
a
lnej
ša







In [4]:
num_examples = 592886 # training set size

nn_output_dim = 13
nn_hdim = 516
batch_size = 16
actual_epoch = 20
num_fake_epoch = 20



conv_input_shape=(10, 252)
othr_input = (150, )

conv_input = Input(shape=conv_input_shape, name='conv_input')
# x_conv = Conv1D(43, (3), padding='same', activation='relu')(conv_input)
x_conv = Conv1D(200, (2), padding='same', activation='relu')(conv_input)
x_conv = MaxPooling1D(pool_size=2)(x_conv)
x_conv = Flatten()(x_conv)
# x_conv = Dense(516, activation='relu', kernel_constraint=maxnorm(3))(x_conv)

othr_input = Input(shape=othr_input, name='othr_input')
# x_othr = Dense(256, input_dim=167, activation='relu')(othr_input)
# x_othr = Dropout(0.3)(x_othr)
# x_othr = Dense(512, activation='relu')(othr_input)
# x_othr = Dropout(0.3)(x_othr)
# x_othr = Dense(256, activation='relu')(othr_input)

x = concatenate([x_conv, othr_input])
# x = Dense(1024, input_dim=(516 + 256), activation='relu')(x)
x = Dense(256, activation='relu')(x)
x = Dropout(0.3)(x)
x = Dense(256, activation='relu')(x)
x = Dropout(0.3)(x)
x = Dense(256, activation='relu')(x)
x = Dropout(0.3)(x)
x = Dense(nn_output_dim, activation='sigmoid')(x)




model = Model(inputs=[conv_input, othr_input], outputs=x)
opt = optimizers.Adam(lr=1E-4, beta_1=0.9, beta_2=0.999, epsilon=1e-08)
model.compile(loss='binary_crossentropy', optimizer=opt, metrics=[actual_accuracy,])

In [5]:
y_array_train = np.asarray(data.y_train)
accentuation_length_train = (y_array_train > 0).sum()
y_array_test = np.asarray(data.y_test)
accentuation_length_test = (y_array_test > 0).sum()
history = model.fit_generator(data.generator('train', batch_size), 
                              accentuation_length_train/(batch_size * num_fake_epoch), 
                              epochs=actual_epoch*num_fake_epoch, 
                              validation_data=data.generator('test', batch_size), 
                              validation_steps=accentuation_length_test/(batch_size * num_fake_epoch))

Epoch 1/400
1453/1452 [==============================] - 30s - loss: 0.2184 - actual_accuracy: 0.1188 - val_loss: 0.1359 - val_actual_accuracy: 0.5073
Epoch 2/400
1453/1452 [==============================] - 31s - loss: 0.1257 - actual_accuracy: 0.5173 - val_loss: 0.1040 - val_actual_accuracy: 0.6421
Epoch 3/400
1453/1452 [==============================] - 33s - loss: 0.0956 - actual_accuracy: 0.6487 - val_loss: 0.0767 - val_actual_accuracy: 0.7569
Epoch 4/400
1453/1452 [==============================] - 36s - loss: 0.0741 - actual_accuracy: 0.7462 - val_loss: 0.0569 - val_actual_accuracy: 0.8332
Epoch 5/400
1453/1452 [==============================] - 36s - loss: 0.0591 - actual_accuracy: 0.8037 - val_loss: 0.0460 - val_actual_accuracy: 0.8594
Epoch 6/400
1453/1452 [==============================] - 36s - loss: 0.0475 - actual_accuracy: 0.8442 - val_loss: 0.0396 - val_actual_accuracy: 0.8906
Epoch 7/400
1453/1452 [==============================] - 35s - loss: 0.0401 - actual_accuracy:

1453/1452 [==============================] - 37s - loss: 0.0048 - actual_accuracy: 0.9857 - val_loss: 0.0283 - val_actual_accuracy: 0.9539
Epoch 55/400
1453/1452 [==============================] - 39s - loss: 0.0046 - actual_accuracy: 0.9869 - val_loss: 0.0303 - val_actual_accuracy: 0.9499
Epoch 56/400
1453/1452 [==============================] - 40s - loss: 0.0045 - actual_accuracy: 0.9866 - val_loss: 0.0229 - val_actual_accuracy: 0.9564
Epoch 57/400
1453/1452 [==============================] - 37s - loss: 0.0040 - actual_accuracy: 0.9884 - val_loss: 0.0236 - val_actual_accuracy: 0.9575
Epoch 58/400
1453/1452 [==============================] - 37s - loss: 0.0044 - actual_accuracy: 0.9879 - val_loss: 0.0252 - val_actual_accuracy: 0.9604
Epoch 59/400
1453/1452 [==============================] - 40s - loss: 0.0043 - actual_accuracy: 0.9881 - val_loss: 0.0263 - val_actual_accuracy: 0.9535
Epoch 60/400
1453/1452 [==============================] - 38s - loss: 0.0041 - actual_accuracy: 0.987

1453/1452 [==============================] - 41s - loss: 0.0026 - actual_accuracy: 0.9934 - val_loss: 0.0316 - val_actual_accuracy: 0.9600
Epoch 108/400
1453/1452 [==============================] - 38s - loss: 0.0024 - actual_accuracy: 0.9931 - val_loss: 0.0266 - val_actual_accuracy: 0.9666
Epoch 109/400
1453/1452 [==============================] - 40s - loss: 0.0025 - actual_accuracy: 0.9934 - val_loss: 0.0305 - val_actual_accuracy: 0.9575
Epoch 110/400
1453/1452 [==============================] - 39s - loss: 0.0025 - actual_accuracy: 0.9928 - val_loss: 0.0357 - val_actual_accuracy: 0.9586
Epoch 111/400
1453/1452 [==============================] - 42s - loss: 0.0022 - actual_accuracy: 0.9938 - val_loss: 0.0413 - val_actual_accuracy: 0.9513
Epoch 112/400
1453/1452 [==============================] - 42s - loss: 0.0026 - actual_accuracy: 0.9924 - val_loss: 0.0344 - val_actual_accuracy: 0.9560
Epoch 113/400
1453/1452 [==============================] - 42s - loss: 0.0025 - actual_accuracy:

1453/1452 [==============================] - 42s - loss: 0.0019 - actual_accuracy: 0.9947 - val_loss: 0.0454 - val_actual_accuracy: 0.9528
Epoch 161/400
1453/1452 [==============================] - 41s - loss: 0.0016 - actual_accuracy: 0.9962 - val_loss: 0.0261 - val_actual_accuracy: 0.9648
Epoch 162/400
1453/1452 [==============================] - 43s - loss: 0.0018 - actual_accuracy: 0.9948 - val_loss: 0.0372 - val_actual_accuracy: 0.9626
Epoch 163/400
1453/1452 [==============================] - 40s - loss: 0.0016 - actual_accuracy: 0.9957 - val_loss: 0.0339 - val_actual_accuracy: 0.9615
Epoch 164/400
1453/1452 [==============================] - 39s - loss: 0.0022 - actual_accuracy: 0.9949 - val_loss: 0.0364 - val_actual_accuracy: 0.9597
Epoch 165/400
1453/1452 [==============================] - 41s - loss: 0.0016 - actual_accuracy: 0.9953 - val_loss: 0.0351 - val_actual_accuracy: 0.9648
Epoch 166/400
1453/1452 [==============================] - 43s - loss: 0.0021 - actual_accuracy:

1453/1452 [==============================] - 42s - loss: 0.0019 - actual_accuracy: 0.9954 - val_loss: 0.0362 - val_actual_accuracy: 0.9575
Epoch 214/400
1453/1452 [==============================] - 44s - loss: 0.0019 - actual_accuracy: 0.9946 - val_loss: 0.0381 - val_actual_accuracy: 0.9597
Epoch 215/400
1453/1452 [==============================] - 44s - loss: 0.0015 - actual_accuracy: 0.9961 - val_loss: 0.0395 - val_actual_accuracy: 0.9575
Epoch 216/400
1453/1452 [==============================] - 45s - loss: 0.0014 - actual_accuracy: 0.9961 - val_loss: 0.0357 - val_actual_accuracy: 0.9597
Epoch 217/400
1453/1452 [==============================] - 44s - loss: 0.0016 - actual_accuracy: 0.9953 - val_loss: 0.0323 - val_actual_accuracy: 0.9608
Epoch 218/400
1453/1452 [==============================] - 46s - loss: 0.0015 - actual_accuracy: 0.9958 - val_loss: 0.0411 - val_actual_accuracy: 0.9593
Epoch 219/400
1453/1452 [==============================] - 45s - loss: 0.0015 - actual_accuracy:

1453/1452 [==============================] - 42s - loss: 0.0013 - actual_accuracy: 0.9963 - val_loss: 0.0404 - val_actual_accuracy: 0.9553
Epoch 267/400
1453/1452 [==============================] - 44s - loss: 0.0014 - actual_accuracy: 0.9961 - val_loss: 0.0415 - val_actual_accuracy: 0.9597
Epoch 268/400
1453/1452 [==============================] - 44s - loss: 0.0016 - actual_accuracy: 0.9958 - val_loss: 0.0305 - val_actual_accuracy: 0.9604
Epoch 269/400
1453/1452 [==============================] - 44s - loss: 0.0014 - actual_accuracy: 0.9963 - val_loss: 0.0351 - val_actual_accuracy: 0.9608
Epoch 270/400
1453/1452 [==============================] - 45s - loss: 0.0016 - actual_accuracy: 0.9963 - val_loss: 0.0320 - val_actual_accuracy: 0.9637
Epoch 271/400
1453/1452 [==============================] - 48s - loss: 0.0015 - actual_accuracy: 0.9959 - val_loss: 0.0467 - val_actual_accuracy: 0.9542
Epoch 272/400
1453/1452 [==============================] - 44s - loss: 0.0015 - actual_accuracy:

1453/1452 [==============================] - 46s - loss: 0.0014 - actual_accuracy: 0.9963 - val_loss: 0.0405 - val_actual_accuracy: 0.9586
Epoch 320/400
1453/1452 [==============================] - 47s - loss: 0.0012 - actual_accuracy: 0.9974 - val_loss: 0.0375 - val_actual_accuracy: 0.9597
Epoch 321/400
1453/1452 [==============================] - 44s - loss: 0.0013 - actual_accuracy: 0.9970 - val_loss: 0.0346 - val_actual_accuracy: 0.9648
Epoch 322/400
1453/1452 [==============================] - 46s - loss: 0.0013 - actual_accuracy: 0.9968 - val_loss: 0.0426 - val_actual_accuracy: 0.9586
Epoch 323/400
1453/1452 [==============================] - 45s - loss: 0.0013 - actual_accuracy: 0.9970 - val_loss: 0.0415 - val_actual_accuracy: 0.9571
Epoch 324/400
1453/1452 [==============================] - 45s - loss: 0.0012 - actual_accuracy: 0.9969 - val_loss: 0.0419 - val_actual_accuracy: 0.9575
Epoch 325/400
1453/1452 [==============================] - 48s - loss: 0.0013 - actual_accuracy:

1453/1452 [==============================] - 46s - loss: 0.0011 - actual_accuracy: 0.9973 - val_loss: 0.0453 - val_actual_accuracy: 0.9615
Epoch 373/400
1453/1452 [==============================] - 46s - loss: 9.2198e-04 - actual_accuracy: 0.9973 - val_loss: 0.0451 - val_actual_accuracy: 0.9597
Epoch 374/400
1453/1452 [==============================] - 46s - loss: 0.0016 - actual_accuracy: 0.9959 - val_loss: 0.0353 - val_actual_accuracy: 0.9626
Epoch 375/400
1453/1452 [==============================] - 46s - loss: 0.0013 - actual_accuracy: 0.9968 - val_loss: 0.0318 - val_actual_accuracy: 0.9648
Epoch 376/400
1453/1452 [==============================] - 44s - loss: 0.0012 - actual_accuracy: 0.9969 - val_loss: 0.0348 - val_actual_accuracy: 0.9633
Epoch 377/400
1453/1452 [==============================] - 45s - loss: 9.2538e-04 - actual_accuracy: 0.9975 - val_loss: 0.0391 - val_actual_accuracy: 0.9640
Epoch 378/400
1453/1452 [==============================] - 45s - loss: 0.0013 - actual_a

In [27]:
name = '20_epoch'
model.save(name + '.h5')
output = open(name + '_history.pkl', 'wb')
pickle.dump(history.history, output)
output.close()

In [15]:
# dictionary, max_word, max_num_vowels, content, vowels, accetuated_vowels = create_dict()
# train_content, test_content, validate_content = split_content(content, 0.2, '../../internal_representations/inputs/content_shuffle_vector.h5')
# feature_dictionary = create_feature_dictionary()
# syllable_dictionary = create_syllables_dictionary(content, vowels)
# max_syllable = get_max_syllable(syllable_dictionary)


content = data._read_content('../../../data/SlovarIJS_BESEDE_utf8.lex')
dictionary, max_word, max_num_vowels, vowels, accented_vowels = data._create_dict(content)
feature_dictionary = data._create_feature_dictionary()
syllable_dictionary = data._create_syllables_dictionary(content, vowels)

In [8]:
%run ../../../prepare_data.py
# generate_X_and_y(dictionary, max_word, max_num_vowels, content, vowels, accetuated_vowels, feature_dictionary)